In [1]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.eprbulletsafsc.com/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
from collections import defaultdict


anchor_tags = soup.find_all('a')
required_urls = defaultdict(list)
for tag in anchor_tags:
    href = tag['href']
    tag_split = tag.text.split(' ')
    category = tag_split[0]
    sub_category_title = ' '.join(tag_split[1:])

    cat_prefix = category[0] + 'XXXX'

    linked_page_url = url + "/" + href

    if not category[0].isdigit(): 
        required_urls['General'].append(['General', linked_page_url, category + ' ' + sub_category_title])
    else:
        required_urls[cat_prefix].append([category, linked_page_url, sub_category_title])

print(required_urls)
# print(required_urls['General'])

defaultdict(<class 'list'>, {'General': [['General', 'https://www.eprbulletsafsc.com//aadd.htm', 'AADDEPR Bullets'], ['General', 'https://www.eprbulletsafsc.com//adverse-actions.htm', 'AdverseActions EPR Bullets'], ['General', 'https://www.eprbulletsafsc.com//ABDR.htm', 'AircraftBattle Damage Repair (ABDR)'], ['General', 'https://www.eprbulletsafsc.com//adopt-a-highway.htm', 'Adopta Highway Program'], ['General', 'https://www.eprbulletsafsc.com//adpe.htm', 'ADPEMonitor'], ['General', 'https://www.eprbulletsafsc.com//advisory-council.htm', 'AdvisoryCouncil'], ['General', 'https://www.eprbulletsafsc.com//airshow.htm', 'Airshowbullets'], ['General', 'https://www.eprbulletsafsc.com//award.htm', 'Awards'], ['General', 'https://www.eprbulletsafsc.com//booster-club.htm', 'BoosterClub'], ['General', 'https://www.eprbulletsafsc.com//BTZ.htm', 'BTZExamples'], ['General', 'https://www.eprbulletsafsc.com//building-custodian.htm', 'BuildingCustodian'], ['General', 'https://www.eprbulletsafsc.com//c

In [3]:
res = []
required_count = 10
for category in required_urls:
    for sub_category, url, sub_category_title in required_urls[category]:
        print(f"Processing {sub_category} - {sub_category_title}")
        linked_page = requests.get(url)
        linked_soup = BeautifulSoup(linked_page.content, 'html.parser')
        divSoup = linked_soup.find("div", {"class" : "main"})
        ps = divSoup.findAll('p')

        count = 0
        for p in ps:
            splits = p.text.split('\n')

            for line in splits:
                if line != '': # Not having a check somehow leads to duplicates
                    res.append([category, sub_category, sub_category_title, line[2:]])
                    count += 1
                if count == required_count: break
            if count == required_count: break

Processing General - AADDEPR Bullets
Processing General - AdverseActions EPR Bullets
Processing General - AircraftBattle Damage Repair (ABDR)
Processing General - Adopta Highway Program
Processing General - ADPEMonitor
Processing General - AdvisoryCouncil
Processing General - Airshowbullets
Processing General - Awards
Processing General - BoosterClub
Processing General - BTZExamples
Processing General - BuildingCustodian
Processing General - CareerAssistance Advisor (CAA)
Processing General - CCAFExamples
Processing General - CDCs
Processing General - CEM/Superintendent
Processing General - Coinedfor Performance
Processing General - ControlledArea
Processing General - CFC/Fundraisers
Processing General - Changeof Command/Parade
Processing General - Chaplain'sAssistant
Processing General - CommandChief Executive Assistant
Processing General - CommunityService
Processing General - COMSECCustodian
Processing General - Contracting
Processing General - COVID-19
Processing General - CPR
Proc

In [ ]:
import pandas as pd

# for key in res:
    # print(len(res[key]))

df = pd.DataFrame(res, columns=['Category', 'Sub Category', 'Sub Category Title', 'Data']).explode('Data').reset_index(drop=True)
print(df)
df.to_excel('output.xlsx', index=False)